#우울증상 발화문 분류 모델
사용모델: KcELECTRA(이준범, 2021)
아래 영화리뷰 감정분석 구현 코드를 토대로, 프로젝트에 맞게 일부 수정한 코드입니다. 코드 출처: https://velog.io/@seoyeon96/PLM%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-%ED%95%9C%EA%B5%AD%EC%96%B4-%ED%98%90%EC%98%A4-%ED%91%9C%ED%98%84-%ED%83%90%EC%A7%80-6.-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%A6%9D%EA%B0%95

In [ ]:
KcELECTRA 인용 표기
@misc{lee2021kcelectra,
  author = {Junbum Lee},
  title = {KcELECTRA: Korean comments ELECTRA},
  year = {2021},
  publisher = {GitHub},
  journal = {GitHub repository},
  howpublished = {\url{https://github.com/Beomi/KcELECTRA}}
}

In [ ]:
# 코랩 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.8 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/2023_연구학점/jihee/KcELECTRA

/content/drive/.shortcut-targets-by-id/1d17tbNKxtVyvtWhPZgfbdMPdzKw_uZyc/2023_연구학점/jihee/KcELECTRA


In [ ]:
# 패키지 import
import import_ipynb
import copy
import torch
import tqdm
import argparse
import utils

from utils import mask_tokens

from typing import Union
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM

importing Jupyter notebook from utils.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.1 MB/s eta 0:00:00


# New Section

In [ ]:
# 모델 import
def load_tuned_model(args:argparse.Namespace):
    if (torch.cuda.is_available()) and (args.dev_num>=0) and (args.dev_num < torch.cuda.device_count()):
        dev = "cuda:{}".format(args.dev_num)
    else:
        dev = "cpu"
        
    model = AutoModelForMaskedLM.from_pretrained(args.tuned_model_path)
    tokenizer = AutoTokenizer.from_pretrained(args.tuned_model_path)

    model.to(dev)
    return model, tokenizer, dev

# tokenizer import
def tokenize(tokenizer:AutoTokenizer, sent:str):
    encoded_dict = tokenizer(
        sent,
        add_special_tokens = True,
        return_attention_mask = True,
        return_tensors = "pt"
    )
    input_id, attention_mask = encoded_dict.input_ids, encoded_dict.attention_mask

    return input_id, attention_mask

def is_same_token_type(org_token:str, candidate:str) -> bool:
    '''
    후보 필터링 조건을 만족하는지 확인
    - 후보와 원 토큰의 타입을 문장부호와 일반 토큰으로 나누어 같은 타입에 속하는지 확인
    '''
    res = False
    if org_token[0]=="#" and org_token[2:].isalpha()==candidate.isalpha():
        res = True
    elif candidate[0]=="#" and org_token.isalpha()==candidate[2:].isalpha():
        res = True
    elif candidate[0]=="#" and org_token[0]=="#" and org_token[2:].isalpha()==candidate[2:].isalpha():
        res = True
    elif org_token.isalpha()==candidate.isalpha() and (candidate[0]!="#" and org_token[0]!="#"):
        res = True

    return res

def candidate_filtering(tokenizer:AutoTokenizer,
                        input_ids:list,
                        idx:int,
                        org:int,
                        candidates:Union[list, torch.Tensor]) -> int:
    '''
    후보 필터링 조건에 만족하는 최적의 후보 선택
    1. 원래 토큰과 후보 토큰이 같은 타입(is_same_token_type 참고)
    2. 현 위치 앞 혹은 뒤에 동일한 토큰이 있지 않음
    '''

    org_token = tokenizer.convert_ids_to_tokens([org])[0]
    candidate_tokens = tokenizer.convert_ids_to_tokens(candidates.cpu().tolist())

    for rank, token in enumerate(candidate_tokens):
        if org_token!=token and is_same_token_type(org_token, token):
            if input_ids[idx-1]==candidates[rank] or input_ids[idx+1]==candidate_tokens[rank]:
                continue
            return candidates[rank]

    return org

def augment_one_sent(model:AutoModelForMaskedLM,
                    tokenizer:AutoTokenizer,
                    sent:str,
                    dev:Union[str, torch.device],
                    args:Union[argparse.Namespace, dict]) -> str:
    '''
    한 문장에 랜덤으로 마스킹을 적용하여 새로운 문장을 생성(증강)

    args:
        model(AutoModelForMaskedLM)     : finetuned model
        tokenizer(AutoTokenizer)
        sent(str)                       : 증강할 문장
        dev(str or torch.device)
        args(argparse.Namespace)
            - k(int, default=5) : 사용할 후보의 개수. k개의 후보 적절한 토큰이 없을 경우 원래 토큰 그대로 유지
            - threshold(float, default=0.95) : 확률 필터링에 사용할 임계치.
                                               마스크에 대해서 특정 후보 토큰을 생성할 확률이 임계치보다 클 경우에는 별도의 필터링 없이 후보를 그대로 사용.
           -  mlm_prob(float, default=0.15) : 마스킹 비율
        
    return:
        (str) : 증강 문장
    '''

    if type(args) == argparse.Namespace:
        k = args.k
        threshold = args.threshold
        mlm_prob = args.mlm_prob
        truncate=True
    else:
        ## type == dict
        k = args["k"]
        threshold = args["threshold"]
        mlm_prob = args["mlm_prob"]

    model.eval()

    input_id, attention_mask  = tokenize(tokenizer, sent)
    org_ids = copy.deepcopy(input_id[0])
    
    masked_input_id, _ = mask_tokens(tokenizer, input_id, mlm_prob, do_rep_random=False)
    while masked_input_id.cpu().tolist()[0].count(tokenizer.mask_token_id) < 1:
        masked_input_id, _ = mask_tokens(tokenizer, input_id, mlm_prob, do_rep_random=False)
    
    with torch.no_grad():
        masked_input_id, attention_mask = masked_input_id.to(dev), attention_mask.to(dev)
        output = model(masked_input_id, attention_mask = attention_mask)
        logits = output["logits"][0]

    copied = copy.deepcopy(masked_input_id.cpu().tolist()[0])
    for i in range(len(copied)):
        if copied[i] == tokenizer.mask_token_id:
            org_token = org_ids[i]
            prob = logits[i].softmax(dim=0)
            probability, candidates = prob.topk(k)
            if probability[0]<threshold:
                res = candidate_filtering(tokenizer, copied, i, org_token, candidates)
            else:
                res = candidates[0]
            copied[i] = res

    copied = tokenizer.decode(copied, skip_special_tokens=True)

    return copied


def batch_augment(model:AutoModelForMaskedLM,
                tokenizer:AutoTokenizer,
                dataset:torch.utils.data.Dataset,
                dev:Union[str, torch.device],
                args:argparse.Namespace) -> str:
    '''
    배치 단위의 문장에 랜덤으로 마스킹을 적용하여 새로운 문장 배치를 생성(증강)

    args:
        model(AutoModelForMaskedLM)
        tokenizer(AutoTokenizer)
        dataset(torch.utils.data.Dataset)
        dev(str or torch.device)
        args(argparse.Namespace)
            - k(int, default=5)
            - threshold(float, default=0.95)
           -  mlm_prob(float, default=0.15)
        
    return:
        (list) : 증강한 문장들의 리스트
    '''

    k = args.k
    threshold = args.threshold
    mlm_prob = args.mlm_prob
    batch_size = args.batch_size

    model.eval()

    augmented_res = []
    dataloader = DataLoader(dataset, batch_size = batch_size)
    for batch in tqdm.tqdm(dataloader):
        #########################################################
        # 인풋 문장에 랜덤으로 마스킹 적용
        input_ids, attention_masks = batch[0], batch[1]
        masked_input_ids, _ = mask_tokens(tokenizer, input_ids, mlm_prob, do_rep_random=False)

        masked_input_ids = masked_input_ids.to(dev)
        attention_masks = attention_masks.to(dev)
        labels = input_ids
        #########################################################

        with torch.no_grad():
            output = model(masked_input_ids, attention_mask = attention_masks)
            logits1 = output["logits"]

        #########################################################
        # 배치 내의 문장 별로 후보 필터링을 적용하고, 결과를 토대로 새로운 문장 생성
        augmented1 = []
        for sent_no in range(len(masked_input_ids)):
            copied = copy.deepcopy(input_ids.cpu().tolist()[sent_no])

            for i in range(len(masked_input_ids[sent_no])):
                if masked_input_ids[sent_no][i] == tokenizer.pad_token_id:
                    break

                if masked_input_ids[sent_no][i] == tokenizer.mask_token_id:
                    org_token = labels.cpu().tolist()[sent_no][i]
                    prob = logits1[sent_no][i].softmax(dim=0)
                    probability, candidates = prob.topk(k)
                    if probability[0]<threshold:
                        res = candidate_filtering(tokenizer, copied, i, org_token, candidates)
                    else:
                        res = candidates[0]
                    copied[i] = res

            copied = tokenizer.decode(copied, skip_special_tokens=True)
            augmented1.append(copied)
        #########################################################
        augmented_res.extend(augmented1)

    return augmented_res

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/2023_연구학점/jihee/EDA_dataset2/symp9/symp9 - Raw.csv',encoding='utf-8')
# raw_data = df[df['Label']== 1]
raw_data = df['Text'].to_list()

In [ ]:
if __name__ == "__main__":
    import random

    # from data_loader import AugmentDataSet

    random.seed(1)

    import easydict

    args = easydict.EasyDict({
        "tuned_model_path" : 'seoyeon96/KcELECTRA-MLM',
        "dev_num" : -1,
        "input_file" : None,
        "batch_size" : 1,
        "mlm_prob" : 0.15,
        "threshold" :0.95,
        "k" : 5
    })
    
    # parser = argparse.ArgumentParser()
    # parser.add_argument("--tuned_model_path", default="seoyeon96/KcELECTRA-MLM", type=str, help="Finetuned model path")
    # parser.add_argument("--dev_num", default=-1, type=int, help="cuda device number")
    # parser.add_argument("--input_file", default=None, type=str)   # 증강을 적용할 데이터
    # parser.add_argument("--batch_size", default=1, type=int)
    # parser.add_argument("--mlm_prob", default=0.15, type=float)
    # parser.add_argument("--threshold", default=0.95, type=float)
    # parser.add_argument("--k", default=5, type=int)

    # args = parser.parse_args()
    model, tokenizer, dev = load_tuned_model(args) 

    if args.batch_size > 1:
        if args.input_file is None:
            raise Exception("input_file is None")
        with open(args.input_file, "r") as f:
            corpus = f.readlines()

        dataset = AugmentDataSet(corpus, tokenizer)
        augmented = batch_augment(model, tokenizer, dataset, dev, args)
    else:
        # augmented_list = []
        # for i in range(3):
        #   for s in raw_data:
        #     try:
        #       augmented = augment_one_sent(model, tokenizer, s, dev, args)
        #     except:
        #       augmented = ""
        #       augmented_list.append(augmented)
        #     else:
        #       augmented_list.append(augmented)
        # print(augmented_list)
        input_sen = input("INPUT = ").strip()
        augmented = augment_one_sent(model, tokenizer, input_sen, dev, args)
        print("OUTPUT = ", augmented)
        print("-"*30)
       
    

INPUT = 남자 키 159cm외모도 볼품없고동성애자.집안 대대로 가난하고성격도 인프피에 내성적이면서 회피성향짙고 남시선의식 많이하고 자기혐오에 피해의식, 전형적인 자의식과잉에 예민하고 히스테릭함. 한마디로 은은하게 성격이 파탄난 인간의 대명사.장애는 없다. 큰 지병도 없다. 부모에게 학대받고 자라지 않았다.학창시절 교우관계 문제도 없었다.현재 나는 만성적인 우울증과 외모집착,강박을 겪고 있다.내가 노력해서 키가 170이 되고, 이성애자가 되고, 성격을 개조할수 있을까? 그건 내 컨트롤 영역 밖이다.'
OUTPUT =  남자 키 159cm외모도 볼품없는동성애자. 집안이 대대로 가난하고성형도 인프랜드에 내성적이면서 회피성향짙고 남시비의식 많이하고 자기혐오에 피해의식, 전형적인 자기식과잉에 예민하고 히스테릭함. 그리고 은은하게 성격이 파탄난 여자의 대명사. 장애는 없다, 심지어 지병도 없다. 부모에게 학대받고 자라지 않았다. 학창시절 교우관계 문제도 없었다. 그래서 나는 만성적인 우울증과 외모 패착, 강박을 겪고 있다. 내가 노력해서 키가 170이 되고, 이성병자가 되고. 성격을 컨트롤할수 있을까? 그건 스스로 컨트롤 영역 밖이다,.
------------------------------


In [ ]:
len(augmented_list)


297

In [ ]:
augmented_data = pd.DataFrame(zip(augmented_list))
augmented_data

,0
0,중3남자가학력대학교홈페이지에 있는 우울증테스트를 해봤는데 심각한 우울증이 예상된 사...
1,"우울증 약으로 약을 복용하고 있고, 이년째 복용중입니다요즘 개인적으로 힘든일이 있어..."
2,20대 초반의 여대생입니다. 자살충동이 자꾸 일어나 너무 괴롭습니다. 지금은 휴학중...
3,층간소음으로 인해 우울증에 시달리다가정신과 치료로도 치유가안되서 자살하게되면신반사망...
4,안녕하세요.. 저 중학교 2학년 인데요 지금 공부 스트레스 때문에 미치겠어요ㅠㅠ 그...
...,...
292,"어른들은 그랬다. 자라면서 상처를 받고, 그 상처가 아물면서 딱지를지고, 그 딱지로..."
293,우울증인것같은데확실하진 않고원래 어렸을 때부터 죽고 싶다는 생각 자주 했었는데그런 ...
294,나는 사랑이 넘치는 사람이다많은 사람들에게 이걸 주고 싶었고 줬다처음엔 돌아오는게 ...
295,난 왜 이렇게까지 자기연민에 빠져서 혼자 행복해버릴까 솔직히 객관적으로 봤을때 내가...


In [ ]:
augmented_data.to_csv('/content/drive/MyDrive/2023_연구학점/jihee/KcELECTRA/augmented/augmented9.csv')